In [1]:
from tensorflow.python.platform import gfile
import tensorflow as tf
import numpy as np
from tqdm import tqdm

In [2]:
wts=np.load('weight.npz')
weights=wts[wts.files[0]]
re_train=True
time_steps=1
n_input =3
num_units=64
n_classes=1
lr =0.0001

In [3]:
#data_x=np.loadtxt('./TapData/data_raw_active.csv',delimiter=",")
data_x=np.loadtxt('./TapData/validation_raw.csv',delimiter=",")
#data_y=np.loadtxt('./TapData/data_y_active.csv',delimiter=",")
data_y=np.loadtxt('./TapData/validation_y.csv',delimiter=",")
preY = np.loadtxt('./TapData/data_preY_3C_fullData.active.csv',delimiter=",")
datay=data_y
data_y=data_y.reshape(len(data_y),1)
data_x=data_x[:,(6-n_input):6].reshape((len(data_x),3))
data_state = np.zeros([1,num_units*2],dtype='float')

#loss_weight = preY*len(preY)/preY.sum()+datay*len(datay)/datay.sum()+1
#loss_weight = loss_weight.reshape([len(loss_weight),1])

In [4]:
data_state.shape

(1, 128)

In [5]:
data_state = np.zeros([1,num_units*2],dtype='float')
predictY=np.zeros([len(data_x)])
GRAPH_PB_PATH='./tmp/model/realLSTM_dataV2_U64_i32_mse0.01_61.12.pb'
#GRAPH_PB_PATH='./realLSTM_dataV2_U64_i20_mse0.00_56.20.pb'
with tf.Session() as sess:
    print("load graph")
    with gfile.FastGFile(GRAPH_PB_PATH,'rb') as f:
        graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')
    graph_nodes=[n for n in graph_def.node]
    '''names = []
    for t in graph_nodes:
        names.append(t.name)
        #print(t.input)
    print(names)'''
    for i in tqdm(range(len(data_x))):
        feed={'input_tensor:0':data_x[i:i+1],
              'input_state:0':data_state,
                     
               }
    
        data_state,_predictY = sess.run(['output_state:0','output:0'],feed_dict=feed)
        #print(data_h.shape)
        #data_c = sess.run(c,feed_dict=feed)
        #print(_predictY)
        predictY[i]=_predictY[0]

load graph
Instructions for updating:
Use tf.gfile.GFile.


100%|████████████████████████████████████████████████████████████████████████████| 4643/4643 [00:01<00:00, 4383.69it/s]


In [6]:
np.savetxt('validation_preY_V2.csv',predictY)